In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#           Machine Learning y Deep Learning
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#        Curva ROC y validación de los modelos ML
# ==========================================================
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [ ]:
# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/fraudes_tarjetascredito.csv'

# Cargar los datos en un DataFrame
df= pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(df.head())

In [ ]:
# Separar características (X) y la variable objetivo (y)
X = df.drop(columns=['fraude','precio_compra'])
y = df['fraude']

In [ ]:
# Feature selection aleatoria (solamente por ilustracion):
X_s = X.drop(columns=random.sample(list(X.columns),2))
print(X_s)

In [ ]:
# Estimacion de un solo modelo
porcentaje_entrenamiento = 
X_train, X_test, y_train, y_test = train_test_split(X_s, y, 
                                                    test_size=1 - porcentaje_entrenamiento)
modelo = RandomForestClassifier(
                n_estimators =  # numero de arboles (10 en este ejemplo)
                )
# Entrenar el modelo:
modelo.fit(X_train, y_train)
# Realizar predicciones:
y_pred = modelo.predict_proba(X_test)[:, 1]  
# Calcular curva ROC y AUC para el fold único
fpr_single, tpr_single, _ = roc_curve(y_test, y_pred)
roc_auc_single = auc(fpr_single, tpr_single)

# Gráfico 1: Curva ROC para un solo conjunto de validación
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(fpr_single, tpr_single, color='darkorange', lw=2, label='Curva ROC (AUC = %0.2f)' % roc_auc_single)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Un solo conjunto de validación')
plt.legend(loc="lower right")

In [ ]:
# Generar múltiples curvas ROC entrenando el modelo varias veces
n_iteraciones =   # Número de repeticiones para obtener diferentes curvas ROC (10 en este ejemplo)
tprs = []
fprs = []
aucs = []

# Para asegurar que todas las curvas ROC tengan la misma longitud
mean_fpr = np.linspace(0, 1, 100)

for i in range(n_iteraciones):
    X_s = X.drop(columns=random.sample(list(X.columns), 2))
    porcentaje_entrenamiento = random.uniform(0.5, 0.9)*100
    X_train, X_test, y_train, y_test = train_test_split(X_s, y, 
                                                    test_size=1 - (porcentaje_entrenamiento/100))
    # Reentrenar el modelo y predecir
    arboles = random.randint(10, 100)  # número de árboles
    modelo = RandomForestClassifier(n_estimators=arboles)
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict_proba(X_test)[:, 1]
    
    # Calcular la curva ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    
    # Interpolar TPR para que tenga la misma longitud
    tpr_interp = np.interp(mean_fpr, fpr, tpr)
    tprs.append(tpr_interp)
    fprs.append(mean_fpr)  # Usar el mismo mean_fpr para todas las iteraciones
    aucs.append(roc_auc)
    
    print(f"Iteracion {i+1}: AUC = {roc_auc:.4f}, muestra de entrenamiento (%) = {porcentaje_entrenamiento:.2f}, arboles en el bosque aleatorio = {arboles:.0f}")
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0 
mean_fpr = np.mean(fprs, axis=0)

In [ ]:
# Calcular el promedio y la desviación estándar del area bajo la curva:
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

# Gráfico del promedio de la curva ROC y su banda de desviación estándar
plt.plot(mean_fpr, mean_tpr, color='b', label=r'Curva ROC Promedio (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=0.8)
std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.2, label=r'$\pm$ 1 std. dev.')

# Graficar la línea base
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curvas ROC - Múltiples repeticiones')
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()